In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.neighbors import KNeighborsRegressor

from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split

from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, BaggingRegressor, AdaBoostRegressor


In [2]:
df = pd.read_csv('../data/train_prepared.csv', index_col='Id')
df.head()

,p1,p2,p3,p4,p5,p6,p7,p8,p9,p10,...,p33,p34,p35,p36,p37,revenue,city_group_fine,is_inline,open_utc,is_big_city
Id,,,,,,,,,,,,,,,,,,,,,
0,4,5.0,4.0,4.0,2,2,5,4,5,5,...,5,5,4,3,4,5653753.0,İstanbul,1,9.321696e+08,1
1,4,5.0,4.0,4.0,1,2,5,5,5,5,...,0,0,0,0,0,6923131.0,Ankara,0,1.202947e+09,1
2,2,4.0,2.0,5.0,2,3,5,5,5,5,...,0,0,0,0,0,2055379.0,other,1,1.362787e+09,0
3,6,4.5,6.0,6.0,4,4,10,8,10,10,...,6,18,12,12,6,2675511.0,other,1,1.328141e+09,0
4,3,4.0,3.0,4.0,2,2,5,5,5,5,...,2,3,4,3,3,4316715.0,other,1,1.241827e+09,0


In [3]:
#Drop city_group_fine
df = df.drop(columns='city_group_fine')
df.head()

,p1,p2,p3,p4,p5,p6,p7,p8,p9,p10,...,p32,p33,p34,p35,p36,p37,revenue,is_inline,open_utc,is_big_city
Id,,,,,,,,,,,,,,,,,,,,,
0,4,5.0,4.0,4.0,2,2,5,4,5,5,...,4,5,5,4,3,4,5653753.0,1,9.321696e+08,1
1,4,5.0,4.0,4.0,1,2,5,5,5,5,...,0,0,0,0,0,0,6923131.0,0,1.202947e+09,1
2,2,4.0,2.0,5.0,2,3,5,5,5,5,...,0,0,0,0,0,0,2055379.0,1,1.362787e+09,0
3,6,4.5,6.0,6.0,4,4,10,8,10,10,...,10,6,18,12,12,6,2675511.0,1,1.328141e+09,0
4,3,4.0,3.0,4.0,2,2,5,5,5,5,...,3,2,3,4,3,3,4316715.0,1,1.241827e+09,0


In [4]:
df.dtypes

p1               int64
p2             float64
p3             float64
p4             float64
p5               int64
p6               int64
p7               int64
p8               int64
p9               int64
p10              int64
p11              int64
p12              int64
p13            float64
p14              int64
p15              int64
p16              int64
p17              int64
p18              int64
p19              int64
p20              int64
p21              int64
p22              int64
p23              int64
p24              int64
p25              int64
p26            float64
p27            float64
p28            float64
p29            float64
p30              int64
p31              int64
p32              int64
p33              int64
p34              int64
p35              int64
p36              int64
p37              int64
revenue        float64
is_inline        int64
open_utc       float64
is_big_city      int64
dtype: object

## Train/Test Split

In [5]:
X = df.drop(columns='revenue')
y = df['revenue']

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# PCA

In [7]:
#Standard scale
ss = StandardScaler()
X_train_sc = ss.fit_transform(X_train)
X_test_sc = ss.transform(X_test)

In [8]:
pca = PCA(n_components=2)
pcs_train = pca.fit_transform(X_train_sc)
pcs_test = pca.transform(X_test_sc)
pc_df_train = pd.DataFrame(pcs_train, index=X_train.index)
pc_df_test = pd.DataFrame(pcs_test, index=X_test.index)

In [9]:
print(len(pc_df_train))
pc_df_train.head()

102


,0,1
Id,,
117,0.303265,-1.490681
101,2.219446,-2.615509
0,1.145602,-2.447281
78,11.069298,1.578056
62,-0.587044,-1.966245


In [10]:
print(len(pc_df_test))
pc_df_test.head()

35


,0,1
Id,,
105,-3.943561,-0.439584
104,0.592860,-2.123870
12,-2.870321,0.003369
26,1.102693,-2.016024
123,-4.083652,-0.539542


# Pipeline

In [11]:
#Make a log transformer
logreg = TransformedTargetRegressor(
    regressor=Lasso(),
    func = np.log,
    inverse_func=np.exp
)

In [12]:
logreg_pipe = Pipeline([
    ('ss', StandardScaler()),
    ('pca', PCA()),
    ('ll', logreg)
])

In [13]:
logreg_params = {
    'pca__n_components': list(range(1,6)),
    'll__regressor__alpha': np.logspace(-4, 2, 60)
}

In [14]:
logreg_gridsearch = GridSearchCV(logreg_pipe,     
                              logreg_params,
                              cv=5,        
                              verbose=1,
                              n_jobs=-2)

In [15]:
logreg_gridsearch.fit(X_train, y_train)

Fitting 5 folds for each of 300 candidates, totalling 1500 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('ss', StandardScaler()), ('pca', PCA()),
                                       ('ll',
                                        TransformedTargetRegressor(func=<ufunc 'log'>,
                                                                   inverse_func=<ufunc 'exp'>,
                                                                   regressor=Lasso()))]),
             n_jobs=-2,
             param_grid={'ll__regressor__alpha': array([1.00000000e-04, 1.26384820e-04, 1.59731228e-04, 2.01876025e-04,
       2.55140652e-04, 3.22459055e-04, 4.07539297e-04, 5.15067808e-...
       4.58159767e-01, 5.79044398e-01, 7.31824222e-01, 9.24914728e-01,
       1.16895182e+00, 1.47737765e+00, 1.86718109e+00, 2.35983347e+00,
       2.98247129e+00, 3.76939098e+00, 4.76393801e+00, 6.02089449e+00,
       7.60949669e+00, 9.61724871e+00, 1.21547425e+01, 1.53617495e+01,
       1.94149195e+01, 2.45375111e+01, 3.10116893e+01, 3.91940677e+01,
       4.953

In [16]:
logreg_gridsearch.best_params_

{'ll__regressor__alpha': 0.11242100350620873, 'pca__n_components': 3}

In [17]:
logreg_gridsearch.best_score_

-0.05718121871674002

In [18]:
logreg_gridsearch.score(X_test, y_test)

-0.007297340652984152

So it looks like linear regression with PCA and a log transform on `revenue` does not do better than the null model!

## No log transform

In [19]:
lin_pipe = Pipeline([
    ('ss', StandardScaler()),
    ('pca', PCA()),
    ('lr', Lasso())
])

In [20]:
lin_params = {
    'pca__n_components': list(range(1,11)),
    'lr__alpha': np.logspace(-4, 2, 60)
}

In [21]:
lin_gridsearch = GridSearchCV(lin_pipe,     
                              lin_params,
                              cv=5,        
                              verbose=1,
                              n_jobs=-2)

In [22]:
lin_gridsearch.fit(X_train, y_train)

Fitting 5 folds for each of 600 candidates, totalling 3000 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('ss', StandardScaler()), ('pca', PCA()),
                                       ('lr', Lasso())]),
             n_jobs=-2,
             param_grid={'lr__alpha': array([1.00000000e-04, 1.26384820e-04, 1.59731228e-04, 2.01876025e-04,
       2.55140652e-04, 3.22459055e-04, 4.07539297e-04, 5.15067808e-04,
       6.50967523e-04, 8.22724134e-04, 1.03979842e-03, 1.31414736e-03,
       1.66088278e-03, 2.09910372e-03,...
       1.16895182e+00, 1.47737765e+00, 1.86718109e+00, 2.35983347e+00,
       2.98247129e+00, 3.76939098e+00, 4.76393801e+00, 6.02089449e+00,
       7.60949669e+00, 9.61724871e+00, 1.21547425e+01, 1.53617495e+01,
       1.94149195e+01, 2.45375111e+01, 3.10116893e+01, 3.91940677e+01,
       4.95353521e+01, 6.26051657e+01, 7.91234262e+01, 1.00000000e+02]),
                         'pca__n_components': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]},
             verbose=1)

In [23]:
lin_gridsearch.best_params_

{'lr__alpha': 100.0, 'pca__n_components': 1}

In [24]:
lin_gridsearch.best_score_

-0.054810497865817

In [25]:
lin_gridsearch.score(X_train, y_train)

0.0022501297574921697

In [26]:
lin_gridsearch.score(X_test, y_test)

-0.010376917578286182

## kNN

In [27]:
knn_pipe = Pipeline([
    ('ss', StandardScaler()),
    ('pca', PCA()),
    ('knn', KNeighborsRegressor())
])

In [28]:
knn_params = {
    'pca__n_components': list(range(1,11)),
    'knn__n_neighbors': [3,5,7,9]
}

In [29]:
knn_gridsearch = GridSearchCV(knn_pipe,     
                              knn_params,
                              cv=5,        
                              verbose=1,
                              n_jobs=-2)

In [30]:
knn_gridsearch.fit(X_train, y_train)

Fitting 5 folds for each of 40 candidates, totalling 200 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('ss', StandardScaler()), ('pca', PCA()),
                                       ('knn', KNeighborsRegressor())]),
             n_jobs=-2,
             param_grid={'knn__n_neighbors': [3, 5, 7, 9],
                         'pca__n_components': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]},
             verbose=1)

In [31]:
knn_gridsearch.best_params_

{'knn__n_neighbors': 5, 'pca__n_components': 4}

In [32]:
knn_gridsearch.best_score_

0.04909548088117597

In [33]:
knn_gridsearch.score(X_train, y_train)

0.35419281288669924

In [34]:
knn_gridsearch.score(X_test, y_test)

-0.06985380402105035

## kNN log transform

In [35]:
knn_lr = TransformedTargetRegressor(
    regressor=KNeighborsRegressor(),
    func = np.log,
    inverse_func=np.exp
)

In [36]:
knn_lr_pipe = Pipeline([
    ('ss', StandardScaler()),
    ('pca', PCA()),
    ('knn', knn_lr)
])

In [37]:
knn_lr_params = {
    'pca__n_components': list(range(1,11)),
    'knn__regressor__n_neighbors': [3,5,7,9]
}

In [38]:
knn_lr_gridsearch = GridSearchCV(knn_lr_pipe,     
                              knn_lr_params,
                              cv=5,        
                              verbose=1,
                              n_jobs=-2)

In [39]:
knn_lr_gridsearch.fit(X_train, y_train)

Fitting 5 folds for each of 40 candidates, totalling 200 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('ss', StandardScaler()), ('pca', PCA()),
                                       ('knn',
                                        TransformedTargetRegressor(func=<ufunc 'log'>,
                                                                   inverse_func=<ufunc 'exp'>,
                                                                   regressor=KNeighborsRegressor()))]),
             n_jobs=-2,
             param_grid={'knn__regressor__n_neighbors': [3, 5, 7, 9],
                         'pca__n_components': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]},
             verbose=1)

In [40]:
knn_lr_gridsearch.best_params_

{'knn__regressor__n_neighbors': 5, 'pca__n_components': 3}

In [41]:
knn_lr_gridsearch.best_score_

0.038481582281517504

In [42]:
knn_lr_gridsearch.score(X_train, y_train)

0.3271879045499848

In [43]:
knn_lr_gridsearch.score(X_test, y_test)

-0.10946596275794152

# Non-PCA Methods

## Gradient Boosting

In [44]:
gboost = GradientBoostingRegressor(random_state=42)

gboost_params = {
    'max_depth': [2, 3, 4],
    'n_estimators': range(10,51,10),
    'learning_rate': [0.1, .3, .5, 1]
}

gb_gs = GridSearchCV(gboost, param_grid=gboost_params)

In [45]:
gb_gs.fit(X_train, y_train)

GridSearchCV(estimator=GradientBoostingRegressor(random_state=42),
             param_grid={'learning_rate': [0.1, 0.3, 0.5, 1],
                         'max_depth': [2, 3, 4],
                         'n_estimators': range(10, 51, 10)})

In [46]:
gb_gs.best_params_

{'learning_rate': 0.1, 'max_depth': 2, 'n_estimators': 10}

In [47]:
gb_gs.best_score_

-0.17400043917769173

In [48]:
gb_gs.score(X_train, y_train)

0.33467817564642854

In [49]:
gb_gs.score(X_test, y_test)

-0.0009466959231283489

### More refined gridsearch

In [50]:
gboost = GradientBoostingRegressor(random_state=42)

gboost_params = {
    'max_depth': [2, 3, 4],
    'n_estimators': [5,6,7,8,9,10,11,12,13,14,15],
    'learning_rate': [0.1, .01, .05]
}

gb_gs1 = GridSearchCV(gboost, param_grid=gboost_params)

In [51]:
gb_gs1.fit(X_train, y_train)

GridSearchCV(estimator=GradientBoostingRegressor(random_state=42),
             param_grid={'learning_rate': [0.1, 0.01, 0.05],
                         'max_depth': [2, 3, 4],
                         'n_estimators': [5, 6, 7, 8, 9, 10, 11, 12, 13, 14,
                                          15]})

In [52]:
gb_gs1.best_params_

{'learning_rate': 0.01, 'max_depth': 4, 'n_estimators': 15}

In [53]:
gb_gs1.best_score_

0.03517901944921953

In [54]:
gb_gs1.score(X_train, y_train)

0.16302590556225738

In [55]:
gb_gs1.score(X_test, y_test)

0.0027394001605182128

### Even more refined gridsearch

In [56]:
gboost = GradientBoostingRegressor(random_state=42)

gboost_params = {
    'max_depth': [2, 3, 4, 5],
    'n_estimators': [10,12,15,17,19,21],
    'learning_rate': [.005, .01, .001]
}

gb_gs2 = GridSearchCV(gboost, param_grid=gboost_params)

In [57]:
gb_gs2.fit(X_train, y_train)

GridSearchCV(estimator=GradientBoostingRegressor(random_state=42),
             param_grid={'learning_rate': [0.005, 0.01, 0.001],
                         'max_depth': [2, 3, 4, 5],
                         'n_estimators': [10, 12, 15, 17, 19, 21]})

In [58]:
gb_gs2.best_params_

{'learning_rate': 0.01, 'max_depth': 4, 'n_estimators': 21}

In [59]:
gb_gs2.best_score_

0.038492075157451365

In [60]:
gb_gs2.score(X_train, y_train)

0.216118411278456

In [61]:
gb_gs2.score(X_test, y_test)

0.003330135177407323

## Other approaches

In [62]:
#Without standard scaling - yes I know this is weird for KNN

regressors = [
    RandomForestRegressor(),
    KNeighborsRegressor(),
    LinearRegression(),
    BaggingRegressor(),
    AdaBoostRegressor()]

for reg in regressors:
    print(reg)
    reg.fit(X_train, y_train)
    test_score = reg.score(X_test, y_test)
    train_score = reg.score(X_train, y_train)
    print(f"CV Score: {cross_val_score(reg, X_train, y_train).mean()}")
    print(f"Test Score: {test_score}")
    print(f"Train Score: {train_score}")

RandomForestRegressor()
CV Score: -0.1476894034883007
Test Score: 0.0686871891465477
Train Score: 0.845166838387543
KNeighborsRegressor()
CV Score: -0.1151804712353891
Test Score: 0.15678741258451556
Train Score: 0.2277745734602642
LinearRegression()
CV Score: -18.083622805807558
Test Score: -0.7830577113389798
Train Score: 0.5439282478628904
BaggingRegressor()
CV Score: -0.17179934120058002
Test Score: -0.07765044849653213
Train Score: 0.7372251413986908
AdaBoostRegressor()
CV Score: -0.3474016467497181
Test Score: 0.024393869340119
Train Score: 0.6855284309529406


In [63]:
#With standard scaling

regressors = [
    RandomForestRegressor(),
    KNeighborsRegressor(),
    LinearRegression(),
    BaggingRegressor(),
    AdaBoostRegressor()]

for reg in regressors:
    reg.fit(X_train_sc, y_train)
    test_score = reg.score(X_test_sc, y_test)
    train_score = reg.score(X_train_sc, y_train)
    print(reg)
    print(f"CV Score: {cross_val_score(reg, X_train_sc, y_train).mean()}")
    print(f"Test Score: {test_score}")
    print(f"Train Score: {train_score}")

RandomForestRegressor()
CV Score: -0.13669049391393714
Test Score: 0.06223604582263986
Train Score: 0.83326715415132
KNeighborsRegressor()
CV Score: -0.05636004084062871
Test Score: 0.04487283888407634
Train Score: 0.35858476672937156
LinearRegression()
CV Score: -18.083624104176558
Test Score: -0.7830577107336791
Train Score: 0.5439282478628904
BaggingRegressor()
CV Score: -0.12197929929683207
Test Score: 0.036955035091154054
Train Score: 0.7422115059311114
AdaBoostRegressor()
CV Score: -0.27363862558090146
Test Score: 0.023702396980586737
Train Score: 0.6938571512410432


In [64]:
X_test.std().sort_values(ascending=False)

open_utc       1.261632e+08
p32            6.240691e+00
p34            6.226536e+00
p30            6.171499e+00
p23            5.762382e+00
p19            5.699034e+00
p36            4.901037e+00
p20            4.133690e+00
p16            4.025549e+00
p31            3.921070e+00
p35            3.820555e+00
p18            3.482248e+00
p14            3.070598e+00
p26            3.022812e+00
p21            2.842652e+00
p1             2.745508e+00
p17            2.711631e+00
p27            2.657872e+00
p24            2.593463e+00
p15            2.527163e+00
p7             2.496384e+00
p25            2.281456e+00
p12            2.048980e+00
p6             1.999159e+00
p10            1.959678e+00
p9             1.959678e+00
p11            1.956674e+00
p28            1.926921e+00
p8             1.875091e+00
p37            1.822824e+00
p29            1.643040e+00
p2             1.637276e+00
p33            1.495090e+00
p22            1.278786e+00
p13            1.076291e+00
p5             1.067

## Trying only using one feature

### `open_utc`

In [65]:
X_train_sm = X_train[['open_utc']]
X_test_sm = X_test[['open_utc']]

In [66]:
regressors = [
    KNeighborsRegressor(),
    LinearRegression()]

for reg in regressors:
    print(reg)
    print(f"CV Score: {cross_val_score(reg, X_train_sm, y_train).mean()}")
    reg.fit(X_train_sm, y_train)
    train_score = reg.score(X_train_sm, y_train)
    test_score = reg.score(X_test_sm, y_test)
    print(f"Test Score: {test_score}")
    print(f"Train Score: {train_score}")

KNeighborsRegressor()
CV Score: -0.1151804712353891
Test Score: 0.15444415230013764
Train Score: 0.2277745734602642
LinearRegression()
CV Score: -0.038213258082492874
Test Score: 0.1466182185478504
Train Score: 0.061107650524979795


# Null model

In [67]:
X_train.head()

,p1,p2,p3,p4,p5,p6,p7,p8,p9,p10,...,p31,p32,p33,p34,p35,p36,p37,is_inline,open_utc,is_big_city
Id,,,,,,,,,,,,,,,,,,,,,
117,3,5.0,4.0,5.0,3,3,5,4,4,5,...,3,3,2,3,4,3,1,1,1.135642e+09,1
101,3,4.0,5.0,4.0,2,2,5,4,5,5,...,5,5,5,4,5,5,4,1,1.347149e+09,0
0,4,5.0,4.0,4.0,2,2,5,4,5,5,...,3,4,5,5,4,3,4,1,9.321696e+08,1
78,9,7.5,6.0,7.5,8,6,10,10,8,8,...,9,20,4,18,12,12,2,1,1.348877e+09,1
62,2,4.0,4.0,4.0,2,3,5,5,5,4,...,1,3,2,3,1,3,3,0,1.289779e+09,0


In [68]:
null_pred = y_train.mean()
null_pred

4415069.94117647

In [69]:
train_preds = [null_pred for _ in range(len(y_train))]
test_preds = [null_pred for _ in range(len(y_test))]

In [70]:
#Training score
metrics.r2_score(y_train, train_preds)

0.0

In [71]:
#Test score
metrics.r2_score(y_test, test_preds)

-0.002248826198974818

# Conclusions

We've learned that, when trying to model restaurant revenue using the features given to us and only ~100 points of training data, it is difficult to create any model that we can be confident makes better predictions than the null model.

This is because, with such a small amount of data, there is simply a very large variance to *almost any  model*.  As we saw, even simple models that use 1 or 2 features can have a large variance from one train/test split to another.  In one split (or one transformation) the model may have a test data R-squared score of .15 or higher; but on another split it may have an R-squared score of -.15 or lower.

Indeed, perhaps one or more of the above models actually *does* perform significantly better than the null model across new data.  However, there is no way for us to *know* that to be true without having a much larger quantity of test data.  For example, perhaps `city_group` is the only feature that is predictive of `revenue`, and we merely "got unlucky" and found that it wasn't terribly predictive *in our small training/test data set*.  In this case, building a model that only uses `city_group` could result in a score of .15 or higher on a Kaggle competition evaluating results with 100,000 points of test data.

Looking at some of the winning Kaggle models, it appears that something like this could easily be going on.  Some of the models are built in extremely long `.ipynb` files, while others are built rather quickly.  The winning models contain a mix of these two types of notebooks; it is likely that the winners just got somewhat lucky and happened to hit upon the few features that matter.